 Copyright © Sorbonne University.

 This source code is licensed under the MIT license found in the LICENSE file
 in the root directory of this source tree.

# Outlook

In this notebook we code the Soft Actor-Critic (SAC) algorithm using BBRL.
This algorithm is described in [this
paper](http://proceedings.mlr.press/v80/haarnoja18b/haarnoja18b.pdf) and [this
paper](https://arxiv.org/pdf/1812.05905.pdf).

To understand this code, you need to know more about [the BBRL interaction
model](https://github.com/osigaud/bbrl/blob/master/docs/overview.md) Then you
should run [a didactical
example](https://github.com/osigaud/bbrl/blob/master/docs/notebooks/03-multi_env_autoreset.student.ipynb)
to see how agents interact in BBRL when autoreset=True.

The algorithm is explained in [this
video](https://www.youtube.com/watch?v=U20F-MvThjM) and you can also read [the
corresponding slides](http://pages.isir.upmc.fr/~sigaud/teach/ps/12_sac.pdf).


# Setting up the environment
We first need to setup the environment
Installs the necessary Python and system libraries

In [2]:
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run

    assert (
        run(["pip", "install", "easypip"]).returncode == 0
    ), "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils>=0.5").setup()

import copy
import os

import torch
import torch.nn as nn
from bbrl.workspace import Workspace
from bbrl.agents import Agent, Agents, TemporalAgent, KWAgentWrapper
from bbrl_utils.algorithms import EpochBasedAlgo
from bbrl_utils.nn import build_mlp, setup_optimizer, soft_update_params
from bbrl_utils.notebook import setup_tensorboard
from omegaconf import OmegaConf
from torch.distributions import (
    Normal,
    Independent,
    TransformedDistribution,
    TanhTransform,
)
import bbrl_gymnasium  # noqa: F401

# Learning environment

## Configuration

The learning environment is controlled by a configuration that define a few
important things as described in the example below. This configuration can
hold as many extra information as you need, the example below is the minimal
one.

```python
params = {
    # This defines the a path for logs and saved models
    "base_dir": "${gym_env.env_name}/myalgo_${current_time:}",

    # The Gymnasium environment
    "gym_env": {
        "env_name": "CartPoleContinuous-v1",
    },

    # Algorithm
    "algorithm": {
        # Seed used for the random number generator
        "seed": 1023,

        # Number of parallel training environments
        "n_envs": 8,
                
        # Minimum number of steps between two evaluations
        "eval_interval": 500,
        
        # Number of parallel evaluation environments
        "nb_evals": 10,

        # Number of epochs (loops)
        "max_epochs": 40000,

        # Number of steps (partial iteration)
        "n_steps": 100,
        
    },
}

# Creates the configuration object, i.e. cfg.algorithm.nb_evals is 10
cfg = OmegaConf.create(params)
```

## The RL algorithm

In this notebook, the RL algorithm is based on `EpisodicAlgo`, that defines
the algorithm environment when using episodes. To use such environment, we
just need to subclass `EpisodicAlgo` and to define two things, namely the
`train_policy` and the `eval_policy`. Both are BBRL agents that, given the
environment state, select the action to perform.

```py
  class MyAlgo(EpisodicAlgo):
      def __init__(self, cfg):
          super().__init__(cfg)

          # Define the train and evaluation policies
          # (the agents compute the workspace `action` variable)
          self.train_policy = MyPolicyAgent(...)
          self.eval_policy = MyEvalAgent(...)

algo = MyAlgo(cfg)
```

The `EpisodicAlgo` defines useful objects:

- `algo.cfg` is the configuration
- `algo.nb_steps` (integer) is the number of steps since the training began
- `algo.logger` is a logger that can be used to collect statistics during training:
    - `algo.logger.add_log("critic_loss", critic_loss, algo.nb_steps)` registers the `critic_loss` value on tensorboard
- `algo.evaluate()` evaluates the current `eval_policy` if needed, and keeps the
agent if it was the best so far (average cumulated reward);
- `algo.visualize_best()` runs the best agent on one episode, and displays the video



Besides, it also defines an `iter_episodes` that allows to iterate over partial
episodes (with `n_steps` from `n_envs` environments):

```python3
  # with partial episodes
  for workspace in algo.iter_partial_episodes():
      # workspace is a workspace containing 50 transitions
      # (with autoreset)
      ...
```

## The SquashedGaussianActor

SAC works better with a Squashed Gaussian actor, which transforms a gaussian
distribution with a $tanh$. The computation of the gradient  uses the
reparametrization trick. Note that our attempts to use a
`TunableVarianceContinuousActor` as we did for instance in the notebook about
PPO completely failed. Such failure is also documented in the [OpenAI spinning
up documentation page about
SAC](https://spinningup.openai.com/en/latest/algorithms/sac.html).

The code of the `SquashedGaussianActor` actor is below.

The fact that we use the reparametrization trick is hidden inside the code of
this distribution. You can read more about the reparametrization trick in at
the following URLs:
- [Goker Erdogan's
  blog](http://gokererdogan.github.io/2016/07/01/reparameterization-trick/)
  which shows the variance of different tricks to compute gradient of
  expectations for $\mathbb{E}(x^2)$ where $x \sim \mathcal{N}(\theta, 1)$

In [3]:
class SquashedGaussianActor(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim, min_std=1e-4):
        """Creates a new Squashed Gaussian actor

        :param state_dim: The dimension of the state space
        :param hidden_layers: Hidden layer sizes
        :param action_dim: The dimension of the action space
        :param min_std: The minimum standard deviation, defaults to 1e-4
        """
        super().__init__()
        self.min_std = min_std
        backbone_dim = [state_dim] + list(hidden_layers)
        self.layers = build_mlp(backbone_dim, activation=nn.ReLU())
        self.backbone = nn.Sequential(*self.layers)
        self.last_mean_layer = nn.Linear(hidden_layers[-1], action_dim)
        self.last_std_layer = nn.Linear(hidden_layers[-1], action_dim)
        self.softplus = nn.Softplus()
        
        # cache_size avoids numerical infinites or NaNs when
        # computing log probabilities
        self.tanh_transform = TanhTransform(cache_size=1)

    def normal_dist(self, obs: torch.Tensor):
        """Compute normal distribution given observation(s)"""
        
        backbone_output = self.backbone(obs)
        mean = self.last_mean_layer(backbone_output)
        std_out = self.last_std_layer(backbone_output)
        std = self.softplus(std_out) + self.min_std
        # Independent ensures that we have a multivariate
        # Gaussian with a diagonal covariance matrix (given as
        # a vector `std`)
        return Independent(Normal(mean, std), 1)

    def forward(self, t, stochastic=True):
        """Computes the action a_t and its log-probability p(a_t| s_t)

        :param stochastic: True when sampling
        """
        normal_dist = self.normal_dist(self.get(("env/env_obs", t)))
        action_dist = TransformedDistribution(normal_dist, [self.tanh_transform])
        if stochastic:
            # Uses the re-parametrization trick
            action = action_dist.rsample()
        else:
            # Directly uses the mode of the distribution
            action = self.tanh_transform(normal_dist.mode)

        log_prob = action_dist.log_prob(action)
        # This line allows to deepcopy the actor...
        self.tanh_transform._cached_x_y = [None, None]
        self.set(("action", t), action)
        self.set(("action_logprobs", t), log_prob)

### Critic agent Q(s,a)

As critics and target critics, SAC uses several instances of ContinuousQAgent
class, as DDPG and TD3. See the [DDPG
notebook](http://master-dac.isir.upmc.fr/rld/rl/04-ddpg-td3.student.ipynb) for
details.

In [4]:
class ContinuousQAgent(Agent):
    def __init__(self, state_dim: int, hidden_layers: list[int], action_dim: int):
        """Creates a new critic agent $Q(s, a)$

        :param state_dim: The number of dimensions for the observations
        :param hidden_layers: The list of hidden layers for the NN
        :param action_dim: The numer of dimensions for actions
        """
        super().__init__()
        self.is_q_function = True
        self.model = build_mlp(
            [state_dim + action_dim] + list(hidden_layers) + [1], activation=nn.ReLU()
        )

    def forward(self, t):
        obs = self.get(("env/env_obs", t))
        action = self.get(("action", t))
        obs_act = torch.cat((obs, action), dim=1)
        q_value = self.model(obs_act).squeeze(-1)
        self.set((f"{self.prefix}q_value", t), q_value)

### Building the complete training and evaluation agents

In the code below we create the Squashed Gaussian actor, two critics and the
corresponding target critics. Beforehand, we checked that the environment
takes continuous actions (otherwise we would need a different code).

In [5]:
# Create the SAC algorithm environment
class SACAlgo(EpochBasedAlgo):
    def __init__(self, cfg):
        super().__init__(cfg)

        obs_size, act_size = self.train_env.get_obs_and_actions_sizes()
        assert (
            self.train_env.is_continuous_action()
        ), "SAC code dedicated to continuous actions"

        # We need an actor
        self.actor = SquashedGaussianActor(
            obs_size, cfg.algorithm.architecture.actor_hidden_size, act_size
        )

        # Builds the critics
        self.critic_1 = ContinuousQAgent(
            obs_size,
            cfg.algorithm.architecture.critic_hidden_size,
            act_size,
        ).with_prefix("critic-1/")
        self.target_critic_1 = copy.deepcopy(self.critic_1).with_prefix(
            "target-critic-1/"
        )

        self.critic_2 = ContinuousQAgent(
            obs_size,
            cfg.algorithm.architecture.critic_hidden_size,
            act_size,
        ).with_prefix("critic-2/")
        self.target_critic_2 = copy.deepcopy(self.critic_2).with_prefix(
            "target-critic-2/"
        )

        # Train and evaluation policies
        self.train_policy = self.actor
        self.eval_policy = KWAgentWrapper(self.actor, stochastic=False)

For the entropy coefficient optimizer, the code is as follows. Note the trick
which consists in using the log of this entropy coefficient. This trick was
taken from the Stable baselines3 implementation of SAC, which is explained in
[this
notebook](https://colab.research.google.com/drive/12LER1_ShWOa_UhOL1nlX-LX_t5KQK9LV?usp=sharing).

Tuning $\alpha$ in SAC is an option. To chose to tune it, the `target_entropy`
argument in the parameters should be `auto`. The initial value is given
through the `entropy_coef` parameter. For any other value than `auto`, the
value of $\alpha$ will stay constant and correspond to the `entropy_coef`
parameter.

In [6]:
def setup_entropy_optimizers(cfg):
    if cfg.algorithm.entropy_mode == "auto":
        # Note: we optimize the log of the entropy coef which is slightly different from the paper
        # as discussed in https://github.com/rail-berkeley/softlearning/issues/37
        # Comment and code taken from the SB3 version of SAC
        log_entropy_coef = nn.Parameter(
            torch.log(torch.ones(1) * cfg.algorithm.init_entropy_coef)
        )
        entropy_coef_optimizer = setup_optimizer(
            cfg.entropy_coef_optimizer, log_entropy_coef
        )
        return entropy_coef_optimizer, log_entropy_coef
    else:
        return None, None

### Compute the critic loss

With the notations of my slides, the equation corresponding to Eq. (5) and (6)
in [this paper](https://arxiv.org/pdf/1812.05905.pdf) becomes:

$$ loss_{Q_{\boldsymbol{\phi}_i}}({\boldsymbol{\theta}}) = {\mathbb{E}}_{(\mathbf{s}_t, \mathbf{a}_t, \mathbf{s}_{t+1}) \sim
\mathcal{D}}\left[\left( r(\mathbf{s}_t, \mathbf{a}_t) + \gamma {\mathbb{E}}_{\mathbf{a} \sim
\pi_{\boldsymbol{\theta}}(.|\mathbf{s}_{t+1})} \left[
\min_{j\in 1,2} \hat{Q}^{\mathrm{target}}_{\boldsymbol{\phi}_j}(\mathbf{s}_{t+1}, \mathbf{a}) - \alpha
\log{\pi_{\boldsymbol{\theta}}(\mathbf{a}|\mathbf{s}_{t+1})} \right] - \hat{Q}_{\boldsymbol{\phi}_i}(\mathbf{s}_t, \mathbf{a}_t) \right)^2
\right] $$

An important information in the above equation and the one about the actor
loss below is the index of the expectations. These indexes tell us where the
data should be taken from. In the above equation, one can see that the index
of the outer expectation is over samples taken from the replay buffer, whereas
in the inner expectation we consider actions from the current actor at the
next state $s_{t+1}$.

Thus, to compute the inner expectation, one needs to determine what actions
the current actor would take in the next state of each sample. This is what
the line

`t_actor(rb_workspace, t=1, n_steps=1, stochastic=True)`

does. The parameter `t=1` (instead of 0) ensures that we consider the next
state $s_{t+1}$.

Once we have determined these actions, we can determine their Q-values and
their log probabilities, to compute the inner expectation.

Note that at this stage, we only determine the log probabilities corresponding
to actions taken at the next time step, by contrast with what we do for the
actor in the `compute_actor_loss(...)` function later on.

Finally, once we have computed the $$
\hat{Q}_{\boldsymbol{\phi}}(\mathbf{s}_{t+1},
\mathbf{a}) $$ for both critics, we take the min and store it into
`post_q_values`. By contrast, the Q-values corresponding to the last term of
the equation are taken from the replay buffer, they are computed in the
beginning of the function by applying the Q agents to the replay buffer
*before* changing the action to that of the current actor.

An important remark is that, if the entropy coefficient $\alpha$ corresponding
to the `ent_coef` variable is set to 0, then we retrieve exactly the critic
loss computation function of the TD3 algorithm. As we will see later, this is
also true of the actor loss computation.

This remark proved very useful in debugging the SAC code. We have set
`ent_coef` to 0 and ensured the behavior was strictly the same as the behavior
of TD3.

Note also that we compute the loss for two critics (initialized
independently), and use two target critics (using the minimum of their
prediction as the basis of the target)

In [18]:
def compute_critic_loss(
    cfg,
    reward: torch.Tensor,
    must_bootstrap: torch.Tensor,
    t_actor: TemporalAgent,
    t_q_agents: TemporalAgent,
    t_target_q_agents: TemporalAgent,
    rb_workspace: Workspace,
    ent_coef: torch.Tensor,
):
    r"""Computes the critic loss for a set of $S$ transition samples

    Args:
        cfg: The experimental configuration
        reward: Tensor (2xS) of rewards
        must_bootstrap: Tensor (2xS) of indicators
        t_actor: The actor agent
        t_q_agents: The critics
        t_target_q_agents: The target of the critics
        rb_workspace: The transition workspace
        ent_coef: The entropy coefficient $\alpha$

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: The two critic losses (scalars)
    """

    # Replay the actor so we get the necessary statistics

    t_q_agents(rb_workspace, t=0, n_steps=1)
    with torch.no_grad():
        t_actor(rb_workspace, t=1, n_steps=1)
        
        action_logprobs = rb_workspace["action_logprobs"]
        t_target_q_agents(rb_workspace, t=1, n_steps=1)
        
    q_value_1, q_value_2, q_values_next_1, q_values_next_2 = rb_workspace["critic-1/q_value", "critic-2/q_value", "target-critic-1/q_value", "target-critic-2/q_value"]

    # Compute temporal difference

    q_values_next = torch.minimum(q_values_next_1[1], q_values_next_2[1])
    esperance_interne = q_values_next - ent_coef*action_logprobs[1]
    
    target = reward[-1] + cfg.algorithm.discount_factor*esperance_interne*must_bootstrap.int()
    
    mse = nn.MSELoss()
    
    critic_loss_1 = mse(q_value_1[0].squeeze(-1), target)
    critic_loss_2 = mse(q_value_2[0].squeeze(-1), target)

    return critic_loss_1, critic_loss_2

### Compute the actor Loss

With the notations of my slides, the equation of the actor loss corresponding
to Eq. (7) in [this paper](https://arxiv.org/pdf/1812.05905.pdf) becomes:

$$ loss_\pi({\boldsymbol{\theta}}) = {\mathbb{E}}_{\mathbf{s}_t \sim
\mathcal{D}}\left[ {\mathbb{E}}_{\mathbf{a}_t\sim
\pi_{\boldsymbol{\theta}}(.|\mathbf{s}_t)} \left[ \alpha
\log{\pi_{\boldsymbol{\theta}}(\mathbf{a}_t|\mathbf{s}_t) -
\hat{Q}_{\boldsymbol{\phi}_{i}}(\mathbf{s}_t,
\mathbf{a}_t)} \right] \right] $$

Note that [the paper](https://arxiv.org/pdf/1812.05905.pdf) mistakenly writes
$Q_\theta(s_t,s_t)$

As for the critic loss, we have two expectations, one over the states from the
replay buffer, and one over the actions of the current actor. Thus we need to
apply again the current actor to the content of the replay buffer.

But this time, we consider the current state, thus we parametrize it with
`t=0` and `n_steps=1`. This way, we get the log probabilities and Q-values at
the current step.

A nice thing is that this way, there is no overlap between the log probability
data used to update the critic and the actor, which avoids having to 'retain'
the computation graph so that it can be reused for the actor and the critic.

This small trick is one of the features that makes coding SAC the most
difficult.

Again, once we have computed the Q values over both critics, we take the min
and put it into `current_q_values`.

As for the critic loss, if we set `ent_coef` to 0, we retrieve the actor loss
function of DDPG and TD3, which simply tries to get actions that maximize the
Q values (by minimizing -Q).

In [ ]:
def compute_actor_loss(
    ent_coef, t_actor: TemporalAgent, t_q_agents: TemporalAgent, rb_workspace: Workspace
):
    r"""
    Actor loss computation
    :param ent_coef: The entropy coefficient $\alpha$
    :param t_actor: The actor agent (temporal agent)
    :param t_q_agents: The critics (as temporal agent)
    :param rb_workspace: The replay buffer (2 time steps, $t$ and $t+1$)
    """

    # Recompute the action with the current actor (at $a_t$)

    t_actor(rb_workspace, t=0, n_steps=1)
    action_logprobs = rb_workspace["action_logprobs"]
    
    t_q_agents(rb_workspace, t=0, n_steps=1)
    q_values_1, q_values_2 = rb_workspace["critic-1/q_value", "critic-2/q_value"]


    # Compute Q-values

    current_q_values = torch.minimum(q_values_1, q_values_2)

    # Compute the actor loss
    
    actor_loss = action_logprobs[0]*ent_coef - current_q_values[0]



    return actor_loss.mean()

## Main training loop

In [24]:
import numpy as np


def run_sac(sac: SACAlgo):
    cfg = sac.cfg
    logger = sac.logger


    # init_entropy_coef is the initial value of the entropy coef alpha.
    ent_coef = cfg.algorithm.init_entropy_coef
    tau = cfg.algorithm.tau_target

    # Creates the temporal actors
    t_actor = TemporalAgent(sac.train_policy)
    t_q_agents = TemporalAgent(Agents(sac.critic_1, sac.critic_2))
    t_target_q_agents = TemporalAgent(Agents(sac.target_critic_1, sac.target_critic_2))

    # Configure the optimizer
    actor_optimizer = setup_optimizer(cfg.actor_optimizer, sac.actor)
    critic_optimizer = setup_optimizer(cfg.critic_optimizer, sac.critic_1, sac.critic_2)
    entropy_coef_optimizer, log_entropy_coef = setup_entropy_optimizers(cfg)


    # If entropy_mode is not auto, the entropy coefficient ent_coef remains
    # fixed. Otherwise, computes the target entropy
    if cfg.algorithm.entropy_mode == "auto":
        # target_entropy is \mathcal{H}_0 in the SAC and aplications paper.
        target_entropy = -np.prod(sac.train_env.action_space.shape).astype(np.float32)

    # Loops over successive replay buffers
    for rb in sac.iter_replay_buffers():
        rb_workspace = rb.get_shuffled(sac.cfg.algorithm.batch_size)
        # Implement the SAC algorithm
        terminated, reward = rb_workspace["env/terminated", "env/reward"]
        # Critic update part #############################
        
        critic_optimizer.zero_grad()
        critic_loss_1, critic_loss_2 = compute_critic_loss(cfg, reward, terminated[1], t_actor, t_q_agents, t_target_q_agents, rb_workspace, ent_coef)
        
        logger.add_log("critic_loss_1", critic_loss_1, sac.nb_steps)
        logger.add_log("critic_loss_2", critic_loss_2, sac.nb_steps)
        critic_loss = critic_loss_1 + critic_loss_2
        print("critic_loss :",critic_loss)
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(
            sac.critic_1.parameters(), cfg.algorithm.max_grad_norm
        )
        torch.nn.utils.clip_grad_norm_(
            sac.critic_2.parameters(), cfg.algorithm.max_grad_norm
        )
        critic_optimizer.step()
        
        # Actor update part #############################

        actor_optimizer.zero_grad()
        actor_loss = compute_actor_loss(ent_coef, t_actor, t_q_agents, rb_workspace)
        print("actor_loss :",actor_loss)
        sac.logger.add_log("actor_loss", actor_loss, sac.nb_steps)

        
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(
            sac.train_policy.parameters(), sac.cfg.algorithm.max_grad_norm
        )
        actor_optimizer.step()

        # Entropy optimizer part
        if entropy_coef_optimizer is not None:
            # See Eq. (17) of the SAC and Applications paper. The log
            # probabilities *must* have been computed when computing the actor
            # loss.
            action_logprobs_rb = rb_workspace["action_logprobs"].detach()
            entropy_coef_loss = -(
                log_entropy_coef.exp() * (action_logprobs_rb + target_entropy)
            ).mean()
            entropy_coef_optimizer.zero_grad()
            entropy_coef_loss.backward()
            entropy_coef_optimizer.step()
            logger.add_log("entropy_coef_loss", entropy_coef_loss, sac.nb_steps)
            logger.add_log("entropy_coef", torch.tensor(ent_coef), sac.nb_steps)

        ####################################################

        # Soft update of target q function
        soft_update_params(sac.critic_1, sac.target_critic_1, tau)
        soft_update_params(sac.critic_2, sac.target_critic_2, tau)

        sac.evaluate()

## Definition of the parameters

In [10]:
params = {
    "save_best": True,
    "base_dir": "${gym_env.env_name}/sac-S${algorithm.seed}_${current_time:}",
    "algorithm": {
        "seed": 1,
        "n_envs": 8,
        "n_steps": 32,
        "buffer_size": 1e6,
        "batch_size": 256,
        "max_grad_norm": 0.5,
        "nb_evals": 16,
        "eval_interval": 2_000,
        "learning_starts": 10_000,
        "max_epochs": 2_000,
        "discount_factor": 0.98,
        "entropy_mode": "auto",  # "auto" or "fixed"
        "init_entropy_coef": 2e-7,
        "tau_target": 0.05,
        "architecture": {
            "actor_hidden_size": [64, 64],
            "critic_hidden_size": [256, 256],
        },
    },
    "gym_env": {"env_name": "CartPoleContinuous-v1"},
    "actor_optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 3e-4,
    },
    "critic_optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 3e-4,
    },
    "entropy_coef_optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 3e-4,
    },
}

## Launching tensorboard to visualize the results

In [ ]:
setup_tensorboard("./outputs/tblogs")

In [31]:
agents = SACAlgo(OmegaConf.create(params))
run_sac(agents)

  0%|          | 0/2000 [00:00<?, ?it/s]

critic_loss : tensor(2.3864, grad_fn=<AddBackward0>)
actor_loss : tensor(0.0716, grad_fn=<MeanBackward0>)
critic_loss : tensor(2.1411, grad_fn=<AddBackward0>)
actor_loss : tensor(0.0092, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.9394, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.0432, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.7463, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.0974, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.5642, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.1510, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.3992, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.2007, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.2367, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.2511, grad_fn=<MeanBackward0>)
critic_loss : tensor(1.0717, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.3054, grad_fn=<MeanBackward0>)
critic_loss : tensor(0.9643, grad_fn=<AddBackward0>)
actor_loss : tensor(-0.3431, grad_fn=<MeanBackward0>)
critic_loss : tensor(0.8019, grad_fn=<A

KeyboardInterrupt: 

In [21]:
# Visualize the best policy
agents.visualize_best()

Video of best agent recorded in outputs\CartPoleContinuous-v1\sac-S1_20250312-141912\best_agent.mp4
Moviepy - Building video C:\Users\simon\OneDrive\Bureau\ANDROIDE\S2\PANDROIDE\Projet\SAC\outputs\CartPoleContinuous-v1\sac-S1_20250312-141912\best_agent.mp4.
Moviepy - Writing video C:\Users\simon\OneDrive\Bureau\ANDROIDE\S2\PANDROIDE\Projet\SAC\outputs\CartPoleContinuous-v1\sac-S1_20250312-141912\best_agent.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\simon\OneDrive\Bureau\ANDROIDE\S2\PANDROIDE\Projet\SAC\outputs\CartPoleContinuous-v1\sac-S1_20250312-141912\best_agent.mp4


## Exercises

- use the same code on the Pendulum-v1 environment. This one is harder to
  tune. Get the parameters from the
  [rl-baseline3-zoo](https://github.com/DLR-RM/rl-baselines3-zoo) and see if
  you manage to get SAC working on Pendulum